In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


In [4]:
import torch
import os
torch.cuda.empty_cache()

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [6]:
from torch.utils.data import Dataset
from torchvision.datasets import CIFAR10
import torchvision.transforms as T


class CIFAR10Dataset(Dataset):
    """CIFAR10 dataset.

    Feature images are automatically flattened.

    Parameters
    ----------
    root : str
        Directory where the actual data is located (or downloaded to).

    train : bool
        If True the training set is returned (60_000 samples). Otherwise
        the validation set is returned (10_000 samples).

    Attributes
    ----------
    tv_dataset : CIFAR10
        Instance of the torchvision `CIFAR10` dataset class.
    """

    def __init__(self, root, train=True, download=True):
      if train:
        transform = T.Compose([
          T.Resize(256),
          T.RandomHorizontalFlip(),
          T.ToTensor(),
          T.Normalize(mean=[0.485, 0.456, 0.486], std=[0.229 , 0.224, 0.225]),
        ]) 
      else:
          transform = T.Compose([
              T.Resize(256),
              T.ToTensor(),
              T.Normalize(mean=[0.485, 0.456, 0.486], std=[0.229 , 0.224, 0.225]),
        ]) 
      self.tv_dataset = CIFAR10(
            root, 
            train=train,
            download=download,
            transform=transform,
        )

    def __len__(self):
        """Get the length of the dataset."""
        return len(self.tv_dataset)

    def __getitem__(self, ix):
        """Get a selected sample.

        Parameters
        ----------
        ix : int
            Index of the sample to get.

        Returns
        -------
        x : torch.Tensor
            Flattened feature tensor of shape `(784,)`.

        y : torch.Tensor
            Scalar representing the ground truth label. Number between 0 and 9.
        """
        return self.tv_dataset[ix]


In [7]:
import math

import torch
import torch.nn as nn
import torch.nn.utils.prune as prune

# результаты исследований показали, что только у сверточных и линейных слоев по 2 или по 3(после обрезки)
            # параметра поэтому их можно фильтровать по этому признаку. Второе наблюдение - глобал прунинг прунит
            # именно те слои, которые ты в него добавляешь из модульлиста. Поэтому осталось
            #  1) сделать module_list со всеми слоями
            #  2) в форварде запустить все через этот module_list
            #  3) чтобы обрезать достаточно добавить слои с 2-3 параметрами в state_dict в parameters_to_prune
            #  4) чтобы скопировать веса достаточно пройтись по всем элементам модульлиста с 2-3 параметрами

class ResNet18(nn.Module):
    def __init__(self, image_channels=3, num_classes=10):
        super(ResNet18, self).__init__()
        layer_list = []
        ds_list = []
        self.relu = nn.ReLU()
        self.avgpool = nn.AvgPool2d(4)
        self.bn16 = nn.BatchNorm2d(16)
        self.bn32 = nn.BatchNorm2d(32)
        self.bn64 = nn.BatchNorm2d(64)
        self.in_planes = 16

        self.conv1 = nn.Conv2d(image_channels, 16, kernel_size=3, stride=1, padding=1, bias=False)
        
        number_of_channels = [16, 32, 64]
        strides_list = [4, 4, 4]
        num_blocks = 3
        for planes, stride in zip(number_of_channels, strides_list):
          strides = [stride] + [1, 1]
          layers = []
          for stride in strides:
              layer_list.append(nn.Conv2d(self.in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False))
              layer_list.append(nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False))
              if stride != 1 or self.in_planes != planes:
                  ds_list.append(nn.Conv2d(self.in_planes, planes, kernel_size=1, stride=stride, bias=False))
              self.in_planes = planes
        
        self.linear = nn.Linear(64, num_classes)

        self.module_list = nn.ModuleList(layer_list)
        self.downsample_list = nn.ModuleList(ds_list)

        self.ptp = []
        for layer in self.module_list:
          self.ptp.append((layer, "weight"))


    def forward(self, x):
        out = self.relu(self.bn16(self.conv1(x)))

        # out = self.layer1(out)
        tmp = self.relu(self.bn16(self.module_list[0](out)))
        tmp = self.bn16(self.module_list[1](tmp))
        tmp += self.bn16(self.downsample_list[0](out))
        out = self.relu(tmp)

        tmp = self.relu(self.bn16(self.module_list[2](out)))
        tmp = self.bn16(self.module_list[3](tmp))
        tmp += out
        out = self.relu(tmp)

        tmp = self.relu(self.bn16(self.module_list[4](out)))
        tmp = self.bn16(self.module_list[5](tmp))
        tmp += out
        out = self.relu(tmp)

        # out = self.layer2(out)
        tmp = self.relu(self.bn32(self.module_list[6](out)))
        tmp = self.bn32(self.module_list[7](tmp))
        tmp += self.bn32(self.downsample_list[1](out))
        out = self.relu(tmp)

        tmp = self.relu(self.bn32(self.module_list[8](out)))
        tmp = self.bn32(self.module_list[9](tmp))
        tmp += out
        out = self.relu(tmp)

        tmp = self.relu(self.bn32(self.module_list[10](out)))
        tmp = self.bn32(self.module_list[11](tmp))
        tmp += out
        out = self.relu(tmp)

        # out = self.layer3(out)
        tmp = self.relu(self.bn64(self.module_list[12](out)))
        tmp = self.bn64(self.module_list[13](tmp))
        tmp += self.bn64(self.downsample_list[2](out))
        out = self.relu(tmp)

        tmp = self.relu(self.bn64(self.module_list[14](out)))
        tmp = self.bn64(self.module_list[15](tmp))
        tmp += out
        out = self.relu(tmp)

        tmp = self.relu(self.bn64(self.module_list[16](out)))
        tmp = self.bn64(self.module_list[17](tmp))
        tmp += out
        out = self.relu(tmp)

        # head
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out


def prune_net(net, prune_ratio=0.3, method="l1"):
    """Prune the whole net.

    Modifies the module in-place. We make an assumption that each
    layer has the bias included.

    Parameters#
    ----------
    net : net
        ResNet18 instance.

    prune_ratio : float
        Number between 0 and 1 representing the percentage of weights
        to prune.

    method : str, {"l1", "random"}
        Pruning method to use.
    """

    parameters_to_prune = []
    for layer in net.module_list:
      parameters_to_prune.append((layer, "weight"))

    prune.global_unstructured(parameters_to_prune,
                          prune.L1Unstructured,
                          amount=prune_ratio)
    


def check_pruned_layer(layer):
    """Check if a module was pruned.

    We require both the bias and the weight to be pruned.

    Parameters
    ----------
    linear : nn.Linear
        Linear module containing a bias.

    Returns
    -------
    bool
        True if the model has been pruned.
    """
    params = {param_name for param_name, _ in layer.named_parameters()}

    return "weight_orig" in params 


def copy_weights_layer(layer_unpruned, layer_pruned):
    assert check_pruned_layer(layer_pruned)
    assert not check_pruned_layer(layer_unpruned)

    with torch.no_grad():
        layer_pruned.weight_orig.copy_(layer_unpruned.weight)


def copy_weights_net(net_unpruned, net_pruned):
    zipped = zip(net_unpruned.module_list, net_pruned.module_list)

    for layer_unpruned, layer_pruned in zipped:
        copy_weights_layer(layer_unpruned, layer_pruned)


In [8]:
import torch
import torch.nn as nn
import tqdm
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader

import wandb


def loop_dataloader(dataloader):
    """Loop infinitely over a dataloader.

    Parameters
    ----------
    dataloader : DataLoader
        DataLoader streaming batches of samples.

    Yields
    ------
    X_batch : torch.Tensor
        Batch of features.

    y_batch : torch.Tensor
        Batch of predictions.
    """
    while True:
        for x in iter(dataloader):
            yield x


def train(model, dataloader_train, loss_inst, optimizer, max_iter=10_000,
          dataloader_val=None, val_freq=500, scheduler=None):
    """Run the training loop.

    Parameters
    ----------
    model : nn.Module
        Neural network (in our case MLP).

    dataloader_train : DataLoader
        Dataloader yielding training samples.

    loss_inst : callable
        Computes the loss when called.

    optimizer : torch.optim.Optimizer
        Instance of an optimizer.

    max_iter : int
        The number of iterations we run the training for
        (= number of graident descent steps).

    dataloader_val : None or DataLoader
        Dataloader yielding validation samples. If provided it will
        also single to us that we want to track metrics.

    val_freq : int
        How often evaluation run.
    """
    global device
    iterable = loop_dataloader(dataloader_train)
    iterable = tqdm.tqdm(iterable, total=max_iter)
    it = 0
    for X_batch, y_batch in iterable:
        if it == max_iter:
            break
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        logit_batch = model(X_batch)

        loss = loss_inst(logit_batch, y_batch)
        if dataloader_val is not None:
            wandb.log({"loss": loss}, step=it)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if scheduler is not None:
          scheduler.step()

        if it % val_freq == 0 and dataloader_val is not None:
            is_equal = []

            for X_batch_val, y_batch_val in dataloader_val:
                X_batch_val = X_batch_val.to(device)
                y_batch_val = y_batch_val.to(device)
                is_equal.append(
                    model(X_batch_val).argmax(dim=-1) == y_batch_val
                )

            is_equal_t = torch.cat(is_equal)
            acc = is_equal_t.sum() / len(is_equal_t)
            wandb.log({"accuracy_val": acc}, step=it)

        it += 1



def experiment(MAX_ITERS, PRUNE_ITERS, PRUNE_RATIOS , RANDOM_STATES, SCHEDULE, LR, project_name):
    args = {"max-iter" : 30000, "batch-size" : 64, "prune-iter" : 1,
          "prune-method" : "l1", "prune-ratio" : 0.2, "val-freq" : 250, "random-state" : 1}
    
    
    for max_iter, prune_iter, prune_ratio, random_state, schedule, lr in zip(MAX_ITERS, PRUNE_ITERS, PRUNE_RATIOS, RANDOM_STATES, SCHEDULE, LR):
            
            os.mkdir("/content/drive/MyDrive/trained_models/prune_ratio_" + str(prune_ratio))
            

            args["max-iter"] = max_iter
            args["prune-iter"] = prune_iter
            args["prune-ratio"] = prune_ratio
            args["random-state"] = random_state

            wandb.init(
                project=project_name,
                entity="bspanfilov",
                config=args,
            )
            wandb.define_metric("accuracy_val", summary="max")

            dataset_train = CIFAR10Dataset(
                "data",
                train=True,
                download=True,
            )
            dataset_val = CIFAR10Dataset(
                "data",
                train=False,
                download=True,
            )

            if args["random-state"] is not None:
                torch.manual_seed(args["random-state"])

            dataloader_train = DataLoader(
                dataset_train, batch_size=args["batch-size"], shuffle=True
            )
            dataloader_val = DataLoader(
                dataset_val, batch_size=args["batch-size"], shuffle=True
            )

            kwargs = dict(
                image_channels=3,
                num_classes=10,
            )

            net = ResNet18(**kwargs).to(device)

            net_copy = ResNet18(**kwargs).to(device)
            net_copy.load_state_dict(net.state_dict())

            loss_inst = nn.CrossEntropyLoss()
            optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9)
            if schedule:
              scheduler = MultiStepLR(optimizer, milestones=[20000,25000], gamma=0.1)
            else:
              scheduler = None

            # Train and prune loop
            if args["prune-ratio"] > 0:
                per_round_prune_ratio = 1 - (1 - args["prune-ratio"]) ** (1 / args["prune-iter"])

                per_round_max_iter = int(args["max-iter"] / args["prune-iter"])

                for prune_it in range(args["prune-iter"]):
                    train(
                        net,
                        dataloader_train,
                        loss_inst,
                        optimizer,
                        max_iter=per_round_max_iter,
                    )
                    prune_net(net, per_round_prune_ratio, method=args["prune-method"])
                    torch.save(net, "/content/drive/MyDrive/trained_models/prune_ratio_" + str(prune_ratio) + "/schedule_" + str(int(schedule)) + "_prune_it_" + str(prune_it))

                    copy_weights_net(net_copy, net)

            # Run actual training with a final pruned network
            train(
                net,
                dataloader_train,
                loss_inst,
                optimizer,
                max_iter=args["max-iter"],
                dataloader_val=dataloader_val,
                val_freq=args["val-freq"],
                scheduler=scheduler,
            )

            torch.save(net, "/content/drive/MyDrive/trained_models/prune_ratio_" + str(prune_ratio) + "/schedule_" + str(int(schedule)) + "_pruned")



In [1]:
# тут просто настройки, с которыми я как раз запускал эксперименты
MAX_ITERS=[30000, 30000]
PRUNE_ITERS=[5, 5]
PRUNE_RATIOS=[0.9, 0.9]
SCHEDULE = [False,True]
LR = [0.03, 0.03]
RANDOM_STATES=[2, 2]

experiment(MAX_ITERS, PRUNE_ITERS, PRUNE_RATIOS, RANDOM_STATES, SCHEDULE, LR, "ResNet18 + CIFAR10")

NameError: ignored

In [ ]:
def interpolation(alpha: float, first: ResNet18, second: ResNet18) -> ResNet18:
  kwargs = dict(
      image_channels=3,
      num_classes=10,
  )

  ans = ResNet18(**kwargs)

  print(ans.module_list[0].weight[0][0])

  ans.conv1.weight[0] = alpha * first.conv1.weight[0] + (1 - alpha) * second.conv1.weight[0]

  for i in range(len(ans.module_list)):
    ans.module_list[i].weight[0] = (alpha * (first.module_list[i].weight_orig[0] * first.module_list[i].weight_mask[0]) + \
                                (1. - alpha) * (second.module_list[i].weight_orig[0] * second.module_list[i].weight_mask[0]))

  for i in range(len(ans.downsample_list)):
    ans.downsample_list[0].weight[0] = alpha * first.downsample_list[0].weight[0] + (1 - alpha) * second.downsample_list[0].weight[0]

  ans.linear.weight[0] = alpha * first.linear.weight[0] + (1 - alpha) * second.linear.weight[0]

  return ans
